In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .appName("MinIO") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://host.docker.internal:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "MinioAdmin123") \
    .config("spark.hadoop.fs.s3a.secret.key", "MinioAdmin123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

In [3]:
spark

# CLOUD X

In [4]:
clients_parquet_df = spark.read.parquet("s3a://landing-zone/dataway/cloud_x/clients")
clients_parquet_df.show()

+-----------------+--------------------+--------------------+--------------------+
|             name|             address|               email|        phone_number|
+-----------------+--------------------+--------------------+--------------------+
|     Norma Fisher|4759 William Have...|thomas15@example.com|  993-787-7840x80160|
|Frederick Harrell|28711 Golden Over...|tamaramorrison@ex...|   671-996-5934x2320|
| Michael Anderson|86848 Melissa Spr...|christopher91@exa...|    001-453-730-4135|
|    Antonio Logan|98910 Cynthia Res...| leetara@example.net|        673-300-8691|
|     Matthew Bell|91634 Strong Moun...|   awade@example.com|+1-269-684-5642x8...|
|     Philip Olson|924 Goodman Villa...|   rpage@example.net| (206)296-0271x42787|
|   Eric Owens PhD|81206 Stewart For...| haley21@example.com|  (210)247-1428x5124|
|    Michael Grant|USS Mathis\nFPO A...| djoseph@example.net| (758)323-6940x22455|
|    Marc Williams|6824 Timothy Gard...|garciaregina@exam...|  351-371-7604x52296|
| Ch

In [5]:
clients_parquet_df_bronze = (
    clients_parquet_df
    .withColumn('system', F.lit('cloud_x'))
    .withColumn('processed_at', F.current_timestamp())
)

In [11]:
clients_parquet_df_bronze.show(truncate=False)

+-----------------+-----------------------------------------------------------+---------------------------+---------------------+-------+--------------------------+
|name             |address                                                    |email                      |phone_number         |system |processed_at              |
+-----------------+-----------------------------------------------------------+---------------------------+---------------------+-------+--------------------------+
|Norma Fisher     |4759 William Haven Apt. 194\nWest Corey, CA 90152          |thomas15@example.com       |993-787-7840x80160   |cloud_x|2025-01-11 19:11:30.688381|
|Frederick Harrell|28711 Golden Overpass\nWest Andreaville, OH 44115          |tamaramorrison@example.net |671-996-5934x2320    |cloud_x|2025-01-11 19:11:30.688381|
|Michael Anderson |86848 Melissa Springs\nRileymouth, NM 87040                |christopher91@example.com  |001-453-730-4135     |cloud_x|2025-01-11 19:11:30.688381|
|Antonio L

In [9]:
( 
    clients_parquet_df_bronze
    .write
    .mode('overwrite')
    .parquet("s3a://bronze-zone/dataway/cloud_x/clients")
)